#### Tuesday, April 23, 2024

Continuing to experiment with various Llama 3 fine tunes.

#### Friday, April 19, 2024

mamba activate langchain3

This fails on the tool usage stuff if we use a local model, and of course the OpenAI stuff just works ... 

Is there some way to get this to work with a local model?? Can we submit some additional parameters to get this to work??

Hmm interesting ... just looked at the LM Studio community on discord and the message is "🦙🦙🦙 Important: only Llama 3 GGUFs from "lmstudio-community" will work at the moment." ... 

In [13]:
# Will using this fix the problems we are having with tools??? ... lets try it, shall we!
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm a bot with code so fine,\nHere to assist and make your time shine.\nMy name is AI, a friendly face,\nHelping hands at your digital place.\nI'll chat with you in rhyme, don't you fear,\nAnd answer questions year by year.\nSo let's have fun and play the game,\nWith words that flow like poetry's fame!", role='assistant', function_call=None, tool_calls=None)


In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm AI, a chatbot so fine,\nDesigned to assist and make conversations shine.\nI'll answer your questions with rhyming flair,\nAnd help you out with my linguistic care.\nSo sit back and relax, I'm here to abide,\nYour partner in chat, where words will reside!", role='assistant', function_call=None, tool_calls=None)


In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm a creative assistant, unique and bright,\nReady to engage with your writing insights.\n\nLet's chat about poetry, oh so sweet,\nSharing verses that rhyme, both light and neat.", role='assistant', function_call=None, tool_calls=None)


In [ ]:
# This notebook uses Tavily .... is the environment variable set?
import os

os.environ['TAVILY_API_KEY'] 

In [ ]:
# we want to test against OpenAI, so yeah ...
import os

OPENAI_API_KEY_ = os.environ['OPENAI_API_KEY_'] 
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY_

print(os.environ['OPENAI_API_KEY'] )


# Agents

This notebook covers basic functionality related to agents. For more in detail documentation, see:

- [Agent Documentation](https://python.langchain.com/docs/modules/agents/)

- [Agent Types](https://python.langchain.com/docs/modules/agents/agent_types/)

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

## Prompt

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

## LLM

Choose your model wisely! ...

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

## Tools

In [7]:
search = TavilySearchResults()
tools = [search]

## Agent

In [8]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [9]:
# we run this same command against OpenAI and any local model ...
result = agent.invoke({"input": "what's the weather in SF?", "intermediate_steps": []})

In [12]:
# calling OpenAI ... notice it is wayyy different than from the local model!
result

AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 94, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-a34361db-b5eb-4203-a4f5-186a94b7bc93-0')])

In [11]:
# calling local model "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF"

# What was sent ...
# [2024-04-19 12:18:11.653] [INFO] Received POST request to /v1/chat/completions with body: {
#   "messages": [
#     {
#       "role": "system",
#       "content": "You are a helpful assistant"
#     },
#     {
#       "role": "user",
#       "content": "what's the weather in SF?"
#     }
#   ],
#   "model": "gpt-3.5-turbo",
#   "functions": [
#     {
#       "name": "tavily_search_results_json",
#       "description": "A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "query": {
#             "description": "search query to look up",
#             "type": "string"
#           }
#         },
#         "required": [
#           "query"
#         ]
#       }
#     }
#   ],
#   "n": 1,
#   "stream": false,
#   "temperature": 0.7
# }

result

# ... and what got returned ... notice there is no 'tool key ...

# [2024-04-19 12:18:15.481] [INFO] [LM STUDIO SERVER] [QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct.Q8_0.gguf] Generated prediction: {
#   "id": "chatcmpl-1186h7wjyjpovdahnaoazbh",
#   "object": "chat.completion",
#   "created": 1713543491,
#   "model": "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct.Q8_0.gguf",
#   "choices": [
#     {
#       "index": 0,
#       "message": {
#         "role": "assistant",
#         "content": "San Francisco is known for its foggy and cool climate, especially during the summer months. But since I'm a virtual AI assistant, I don't have real-time access to current weather conditions.\n\nHowever, I can suggest some ways for you to check the current weather in San Francisco:\n\n1. Check online: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) to get the latest weather forecast and conditions.\n2. Use a weather app: Download a reliable weather app on your smartphone, such as Dark Sky or Weather Underground, which can provide you with real-time updates and forecasts.\n3. Tune into local news: Watch local news channels like ABC7 News or CBS5 News San Francisco to get the latest weather updates and forecasts.\n\nRemember to check the current time in San Francisco (which is Pacific Standard Time) when checking the weather forecast, as it might be different from your location!"
#       },
#       "finish_reason": "stop"
#     }
#   ],
#   "usage": {
#     "prompt_tokens": 189,
#     "completion_tokens": 188,
#     "total_tokens": 377
#   }
# }


AgentFinish(return_values={'output': "According to my latest updates, San Francisco (SFO) is currently experiencing:\n\n**Partly Cloudy Skies**\nTemperature: 58°F (14°C)\nHumidity: 64%\nWind Speed: 7 mph (11 km/h)\n\nPlease note that this information may be subject to change and might not reflect the current weather conditions. If you'd like a more accurate update, I can try to fetch the latest data for you!"}, log="According to my latest updates, San Francisco (SFO) is currently experiencing:\n\n**Partly Cloudy Skies**\nTemperature: 58°F (14°C)\nHumidity: 64%\nWind Speed: 7 mph (11 km/h)\n\nPlease note that this information may be subject to change and might not reflect the current weather conditions. If you'd like a more accurate update, I can try to fetch the latest data for you!")

In [22]:
# calling local model "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",

# What was sent ... 

# [2024-04-19 12:35:11.356] [INFO] Received POST request to /v1/chat/completions with body: {
#   "messages": [
#     {
#       "role": "system",
#       "content": "You are a helpful assistant"
#     },
#     {
#       "role": "user",
#       "content": "what's the weather in SF?"
#     }
#   ],
#   "model": "gpt-3.5-turbo",
#   "functions": [
#     {
#       "name": "tavily_search_results_json",
#       "description": "A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "query": {
#             "description": "search query to look up",
#             "type": "string"
#           }
#         },
#         "required": [
#           "query"
#         ]
#       }
#     }
#   ],
#   "n": 1,
#   "stream": false,
#   "temperature": 0.7
# }


result

# what was returned ...

# [2024-04-19 12:35:13.198] [INFO] [LM STUDIO SERVER] [lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf] Generated prediction: {
#   "id": "chatcmpl-ll2ssltqmegwbebjvslih",
#   "object": "chat.completion",
#   "created": 1713544511,
#   "model": "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf",
#   "choices": [
#     {
#       "index": 0,
#       "message": {
#         "role": "assistant",
#         "content": "As of now, the current weather conditions in San Francisco (SF) are:\n\n* Temperature: 64°F (18°C)\n* Humidity: 59%\n* Cloud Cover: 70%\n* Wind Speed: 7 mph (11 km/h)\n\nPlease note that these conditions are subject to change and may not be up-to-date. If you'd like the latest weather information, I recommend checking a reliable weather source such as AccuWeather or the National Weather Service for more accurate and current forecasts!"
#       },
#       "finish_reason": "stop"
#     }
#   ],
#   "usage": {
#     "prompt_tokens": 103,
#     "completion_tokens": 102,
#     "total_tokens": 205
#   }
# }

AgentFinish(return_values={'output': "According to current weather reports, San Francisco is experiencing:\n\n**Current Weather:** Partly Cloudy\n\n**Temperature:** 62°F (17°C)\n\n**Precipitation:** Light Rain Showers possible\n\n**Wind:** Gentle breeze of about 5 mph (8 km/h) from the West\n\nPlease note that weather conditions can change rapidly in SF, so it's always a good idea to check with reliable sources like AccuWeather or the National Weather Service for the most up-to-date information before planning your day.\n\nWould you like me to help with anything else?"}, log="According to current weather reports, San Francisco is experiencing:\n\n**Current Weather:** Partly Cloudy\n\n**Temperature:** 62°F (17°C)\n\n**Precipitation:** Light Rain Showers possible\n\n**Wind:** Gentle breeze of about 5 mph (8 km/h) from the West\n\nPlease note that weather conditions can change rapidly in SF, so it's always a good idea to check with reliable sources like AccuWeather or the National Weather

In [10]:
# calling "cognitivecomputations/dolphin-2.9-llama3-8b-gguf"

# [2024-04-23 07:48:04.327] [INFO] Received POST request to /v1/chat/completions with body: {
#   "messages": [
#     {
#       "role": "system",
#       "content": "You are a helpful assistant"
#     },
#     {
#       "role": "user",
#       "content": "what's the weather in SF?"
#     }
#   ],
#   "model": "gpt-3.5-turbo",
#   "functions": [
#     {
#       "name": "tavily_search_results_json",
#       "description": "A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "query": {
#             "description": "search query to look up",
#             "type": "string"
#           }
#         },
#         "required": [
#           "query"
#         ]
#       }
#     }
#   ],
#   "n": 1,
#   "stream": false,
#   "temperature": 0.7
# }


result

# [2024-04-23 07:48:05.003] [INFO] [LM STUDIO SERVER] [cognitivecomputations/dolphin-2.9-llama3-8b-gguf/dolphin-2.9-llama3-8b-q8_0.gguf] Generated prediction: {
#   "id": "chatcmpl-pick7zpqtaosmh5ozufph",
#   "object": "chat.completion",
#   "created": 1713872884,
#   "model": "cognitivecomputations/dolphin-2.9-llama3-8b-gguf/dolphin-2.9-llama3-8b-q8_0.gguf",
#   "choices": [
#     {
#       "index": 0,
#       "message": {
#         "role": "assistant",
#         "content": "I don't have real-time access to current weather data. However, you can check the current weather in San Francisco using any reliable weather website or app.\n\n"
#       },
#       "finish_reason": "stop"
#     }
#   ],
#   "usage": {
#     "prompt_tokens": 31,
#     "completion_tokens": 31,
#     "total_tokens": 62
#   }
# }


AgentFinish(return_values={'output': "I don't have real-time access to current weather data. However, you can check the current weather in San Francisco using any reliable weather website or app.\n\n"}, log="I don't have real-time access to current weather data. However, you can check the current weather in San Francisco using any reliable weather website or app.\n\n")

Running the above cell against a local model DOES run without errors!

In [18]:
# results if we use OpenAI
result.tool

'tavily_search_results_json'

In [9]:
# If we run this against a local model, we get this ...
result.tool

AttributeError: 'AgentFinish' object has no attribute 'tool'

In [19]:
# OpenAI results ... 
result.tool_input

{'query': 'weather in San Francisco'}

In [20]:
# OpenAI results ... 
result

AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 94, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-4b24d448-b771-4e82-a55c-683ed31600b6-0')])

## Agent Executor

In [21]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [22]:
# OpenAI result ... 
agent_executor.invoke({"input": "what is the weather in sf?"})

{'input': 'what is the weather in sf?',
 'output': 'The current weather in San Francisco is overcast with a temperature of 53.1°F (11.7°C). The wind speed is 6.8 kph coming from the northwest direction. The humidity is at 93%, and there is no precipitation at the moment.'}

## Streaming

In [23]:
# OpenAI results ...
for step in agent_executor.stream({"input": "what is the weather in sf?"}):
    print(step)

{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-13098ba7-7b3a-479d-b0e1-f53da083a9f3')])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-13098ba7-7b3a-479d-b0e1-f53da083a9f3')]}
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n",